**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [82]:
!pip install keras-flops

In [83]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import np_utils
from keras_flops import get_flops

In [84]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [85]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [86]:
# Преобразуйте метки классов в one_hot формат
y_train = np_utils.to_categorical(y_train, CLASSES)
y_val = np_utils.to_categorical(y_val, CLASSES)

In [166]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [191]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(32, 32, 3)),
    
    tf.keras.layers.Conv2D(18, kernel_size=(3, 3), strides=(2, 2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(14, kernel_size=(3, 3), strides=(2, 2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(CLASSES, activation='softmax'),
])

In [192]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-25 21:47:40.842721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 21:47:40.843084: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-25 21:47:40.843313: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-25 21:47:40.843802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 21:47:40.844169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 21:47:40.844516: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

In [193]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_86 (Conv2D)           (None, 16, 16, 18)        504       
_________________________________________________________________
batch_normalization_134 (Bat (None, 16, 16, 18)        72        
_________________________________________________________________
activation_151 (Activation)  (None, 16, 16, 18)        0         
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 8, 8, 18)          0         
_________________________________________________________________
dropout_70 (Dropout)         (None, 8, 8, 18)          0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 4, 4, 14)          2282      
_________________________________________________________________
batch_normalization_135 (Bat (None, 4, 4, 14)        

In [194]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=2000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [195]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=40
)

Epoch 1/40
391/391 [==============================] - 3s 6ms/step - loss: 3.5796 - accuracy: 0.1676 - val_loss: 3.4809 - val_accuracy: 0.1878
Epoch 2/40
391/391 [==============================] - 2s 4ms/step - loss: 3.0165 - accuracy: 0.2568 - val_loss: 2.9567 - val_accuracy: 0.2785
Epoch 3/40
391/391 [==============================] - 2s 4ms/step - loss: 2.7969 - accuracy: 0.3000 - val_loss: 2.8788 - val_accuracy: 0.3007
Epoch 4/40
391/391 [==============================] - 2s 4ms/step - loss: 2.6505 - accuracy: 0.3262 - val_loss: 2.6665 - val_accuracy: 0.3294
Epoch 5/40
391/391 [==============================] - 2s 5ms/step - loss: 2.5271 - accuracy: 0.3488 - val_loss: 2.6270 - val_accuracy: 0.3376
Epoch 6/40
391/391 [==============================] - 2s 4ms/step - loss: 2.4382 - accuracy: 0.3695 - val_loss: 2.5966 - val_accuracy: 0.3516
Epoch 7/40
391/391 [==============================] - 2s 5ms/step - loss: 2.3491 - accuracy: 0.3902 - val_loss: 2.7909 - val_accuracy: 0.3161
Epoch 